<font color=red> **simplify the below** <font> 

Non-technical description of the neural network usage

It is like when you are trying clear out the small debris in your keyboard - if you are like me, who likes to continue working over lunch, you may find your keyboard in a messy state. Traditionally, one use a shaking method, or if you want to be gentle, a tool-tip to rid the debris. And on the other hand, traditionally the make-up brush is used to make the powder on the face smooth when women are putting on make-ups. But I had this inspiration that we can use the make-up brush to clear the debris in keyboards, thereby using a traditional tool for an innovative use.

The commandeer of neural network in this project is the same. Conventionally it is used to perform fancy operations, such as facial recognition. But if you see it as it is, it is just an approximator of complex logics and functions, and thus we can use it to approximate the OAS calculator and do without the expensive Monte Carlo simulation.

Non-technical description of the Gaussian process

To understand Gaussian process intuitively, think of an alien who is trying to find life on earth. And let's say each landing on earth is very costly for the alien, so their goal is try to land as few as possible, and thus they need to have a strategy on where they land to maximize the likelihood on finding life. So they will need to learn from the information they get from points they already landed. So let's say they have landed on both the north pole and the Sahara Desert. They will need to analyze, north pole is too cold and Sahara is too hot, both not ideal for liquid water to exist. So with a bit of knowledge in geography, they can say let's next land on some place in the middle, should it be closer to Sahara or north pole that depends.

So you can see there is a bit of learning available information and deduction going on. Gaussian process can be thought of as a simple and versatile model to interactively automate that learning, and point us to the next viable points. In the project, it is costly to run the OAS calculator in the input space, and the points in the input space where there the OAS calculator has run tend to have good approximation. So the problem is, we need to find in the input space, the next points to run the OAS calculator on, where the current neural net approximation is doing the poorest. And that is where we make use of the Gaussian process.

Input space - high dimensional
price, volatility, coupon, non-call in years, maturity, tenor values of the base curve

The goal is to approximate the OAS calculator with guarantee of precision in a range of the above inputs, where the range of price, volatility and curve indicate what market environment fits the calibration, while the range of coupon, non-call year and maturity indicate which bonds can use the approximator.

It is clear that a simple gridding and caching approach will quickly run afoul of the curse of dimensionality.

We also tried reducing the dimensionality of base curve by using Nielson-Siegal components

The MLP

We tried single and double-layered MLP, with a tanh layer in between.

Number of neurons is found to be sufficient to be 30-50. The small and simple network does not require us to spend much time in hyperparameter turning (empirically, larger networks have bigger chance of having saddle points)

We find that MLPRegressor in sklearn with adam and lbfgs as optimizer works reasonably well.

The main headache is the accuracy of the true OAS calculator and to sample smartly (see the above point for Gaussian process)

Loss function is defined to be p mean, where the larger the p, the more we avert large error. This is a way to guarantee precision, at least empirically.

The Gaussian process optimization

In each iteration, train the MLP on the given set of data, obtain the errors

Train a Gaussian process on the data points of errors.

Further generate extra points in the input space within range, and see which ones are predicted to be significantly non-zero by the Gaussian process. Add those to the set of data

Repeat, until there is no added data, or the number of data points reach a threshold

Specifying the Gaussian process

The kernel is isotropic, i.e. only depends on the distance between the two input points. The distance of input points are weighted Euclidean, reflecting the belief which dimension the error is likely to be more sensitive on.

To customize the above kernel, needed to override the call method in kernel in sklearn

We do not specify extra noise (i.e. alpha is the defaulted very small value in the API). The implicit assumption is we trust the accuracy of the true OAS calculator we are approximating.

Next steps of refining the Gaussian process

GP is probably reaching the limit of number of data (a few thousand) since we need to invert the covariance matrix in training

GP is probably reaching the limit of the number of features as well: should not be more than a dozen.

Maybe a better logic to judge the significance of errors on the next points are needed; see ‘expected improvement’ in Bayesian optimization: https://towardsdatascience.com/bayesian-optimization-a-step-by-step-approach-a1cb678dd2ec